This example reproduces Figure 1 of Zhu et al [1] and shows how boosting can improve prediction accuracy on a multi-class problem. The classification dataset is constructed by taking a ten-dimensional standard normal distribution and defining three classes separated by nested concentric ten-dimensional spheres such that roughly equal numbers of samples are in each class (quantiles of the \chi^2 distribution).

The performance of the SAMME and SAMME.R [1] algorithms are compared. SAMME.R uses the probability estimates to update the additive model, while SAMME uses the classifications only. As the example illustrates, the SAMME.R algorithm typically converges faster than SAMME, achieving a lower test error with fewer boosting iterations. The error of each algorithm on the test set after each boosting iteration is shown on the left, the classification error on the test set of each tree is shown in the middle, and the boost weight of each tree is shown on the right. 

All trees have a weight of one in the SAMME.R algorithm and therefore are not shown.

[1] J. Zhu, H. Zou, S. Rosset, T. Hastie, “Multi-class AdaBoost”, 2009.


#### New to Plotly?
Plotly's Python library is free and open source! [Get started](https://plot.ly/python/getting-started/) by downloading the client and [reading the primer](https://plot.ly/python/getting-started/).
<br>You can set up Plotly to work in [online](https://plot.ly/python/getting-started/#initialization-for-online-plotting) or [offline](https://plot.ly/python/getting-started/#initialization-for-offline-plotting) mode, or in [jupyter notebooks](https://plot.ly/python/getting-started/#start-plotting-online).
<br>We also have a quick-reference [cheatsheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) (new!) to help you get started!

### Version

In [1]:
import sklearn
sklearn.__version__

'0.18.1'

### Imports

This tutorial imports [make_gaussian_quantiles](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_gaussian_quantiles.html#sklearn.datasets.make_gaussian_quantiles), [AdaBoostClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier), [accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score) and [DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier).

In [2]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

from sklearn.externals.six.moves import zip

from sklearn.datasets import make_gaussian_quantiles
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


### Calculations

In [3]:
X, y = make_gaussian_quantiles(n_samples=13000, n_features=10,
                               n_classes=3, random_state=1)

n_split = 3000

X_train, X_test = X[:n_split], X[n_split:]
y_train, y_test = y[:n_split], y[n_split:]

bdt_real = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),
    n_estimators=600,
    learning_rate=1)

bdt_discrete = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),
    n_estimators=600,
    learning_rate=1.5,
    algorithm="SAMME")

bdt_real.fit(X_train, y_train)
bdt_discrete.fit(X_train, y_train)

real_test_errors = []
discrete_test_errors = []

for real_test_predict, discrete_train_predict in zip(
        bdt_real.staged_predict(X_test), bdt_discrete.staged_predict(X_test)):
    real_test_errors.append(
        1. - accuracy_score(real_test_predict, y_test))
    discrete_test_errors.append(
        1. - accuracy_score(discrete_train_predict, y_test))

n_trees_discrete = len(bdt_discrete)
n_trees_real = len(bdt_real)

# Boosting might terminate early, but the following arrays are always
# n_estimators long. We crop them to the actual number of trees here:
discrete_estimator_errors = bdt_discrete.estimator_errors_[:n_trees_discrete]
real_estimator_errors = bdt_real.estimator_errors_[:n_trees_real]
discrete_estimator_weights = bdt_discrete.estimator_weights_[:n_trees_discrete]


### Plot Results

In [4]:
fig = tools.make_subplots(rows=1, cols=3)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]



In [5]:
samme = go.Scatter(x=range(1, n_trees_discrete + 1),
                   y=discrete_test_errors,
                   showlegend=False,
                   mode='lines',
                   line=dict(color='black', width=1),
                   name='SAMME'
                  )

samme_r = go.Scatter(x=range(1, n_trees_real + 1),
                     y=real_test_errors, 
                     showlegend=False,
                     mode='lines',
                     line=dict(color='black', 
                               dash='dash', width=1),
                     name='SAMME.R')

fig.append_trace(samme, 1, 1)
fig.append_trace(samme_r, 1, 1)
fig['layout']['xaxis1'].update(title='Number of Trees', showgrid=False)
fig['layout']['yaxis1'].update(title='Test Error', showgrid=False)

samme1 = go.Scatter(x=range(1, n_trees_discrete + 1),
                    y=discrete_estimator_errors,
                    showlegend=False,
                    mode='lines',
                    line=dict(color='blue', width=1),
                    name='SAMME')

samme_r1 = go.Scatter(x=range(1, n_trees_real + 1), 
                      y=real_estimator_errors,
                      showlegend=False,
                      mode='lines',
                      line=dict(color='red', width=1),
                      name='SAMME.R')

fig.append_trace(samme1, 1, 2)
fig.append_trace(samme_r1, 1, 2)
fig['layout']['xaxis2'].update(title='Number of Trees', showgrid=False)
fig['layout']['yaxis2'].update(title='Error', showgrid=False,
                               range=[.2,
                                      max(real_estimator_errors.max(),
                                      discrete_estimator_errors.max()) * 1.2])


samme2 = go.Scatter(x=range(1, n_trees_discrete + 1), 
                    y=discrete_estimator_weights,
                    showlegend=False,
                    mode='lines',
                    line=dict(color='blue', width=1),
                    name='SAMME') 
                   
fig.append_trace(samme2, 1, 3)
fig['layout']['xaxis3'].update(title='Number of Trees', showgrid=False)
fig['layout']['yaxis3'].update(title='Weight', showgrid=False)


In [6]:
py.iplot(fig)

### License

Author: 

        Noel Dawe <noel.dawe@gmail.com>

License:

        BSD 3 clause

In [2]:
from IPython.display import display, HTML

display(HTML('<link href="//fonts.googleapis.com/css?family=Open+Sans:600,400,300,200|Inconsolata|Ubuntu+Mono:400,700" rel="stylesheet" type="text/css" />'))
display(HTML('<link rel="stylesheet" type="text/css" href="http://help.plot.ly/documentation/all_static/css/ipython-notebook-custom.css">'))

! pip install git+https://github.com/plotly/publisher.git --upgrade
import publisher
publisher.publish(
    'Multi-class AdaBoosted Decision Trees.ipynb', 'scikit-learn/plot-adaboost-multiclass/', 'Multi-Class AdaBoosted Decision Trees | plotly',
    ' ',
    title = 'Multi-Class AdaBoosted Decision Trees | plotly',
    name = 'Multi-Class AdaBoosted Decision Trees',
    has_thumbnail='true', thumbnail='thumbnail/multiclass-adaboost.jpg', 
    language='scikit-learn', page_type='example_index',
    display_as='ensemble_methods', order=16,
    ipynb= '~Diksha_Gabha/3044')

  Cloning https://github.com/plotly/publisher.git to /tmp/pip-82fWwH-build
  Found existing installation: publisher 0.10
    Uninstalling publisher-0.10:
      Successfully uninstalled publisher-0.10
  Running setup.py install for publisher ... - \ done
